# In this repo you can finetune any sequence classification transformer model on your dataset

In [ ]:
! pip install tensorflow
! pip install nltk
! pip install scikit-learn
! pip install transformers
! pip install datasets
!pip install matplotlib

In [ ]:
import numpy as np
import pandas as pd
import os
from os import listdir
from os.path import isfile, join
import datetime
import time
import torch
import random
import re
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics import f1_score
from sklearn.metrics import confusion_matrix, classification_report
from transformers import AutoModel, AutoTokenizer,BertForSequenceClassification, AdamW, BertConfig,get_linear_schedule_with_warmup
from tensorflow.keras.preprocessing.sequence import pad_sequences
from nltk.tokenize import TweetTokenizer
from sklearn.metrics import precision_recall_fscore_support, classification_report
from torch.utils.data import TensorDataset, DataLoader, RandomSampler, SequentialSampler

In [ ]:
def flat_accuracy(preds, labels):
    pred_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    return np.sum(pred_flat == labels_flat) / len(labels_flat)


def format_time(elapsed):
    elapsed_rounded = int(round((elapsed)))
    return str(datetime.timedelta(seconds=elapsed_rounded))


def normalizeToken(token):
    token = token.strip()
    lowercased_token = token.lower().strip()
    # print(token)
    if token != " ":
        if token.startswith("@"):
            return "@USER"
        elif lowercased_token.startswith("http") or lowercased_token.startswith("www"):
            return "HTTPURL"
        # elif len(token) == 1:
        #     return demojize(token)
        else:
            if token == "’":
                return "'"
            elif token == "…":
                return "..."
            else:
                return token


import re
from nltk.tokenize import TweetTokenizer

def normalizeTweet(tweet):
    tweet = str(tweet)  # Convert to string if tweet is a float
    tok = TweetTokenizer()
    tokens = tok.tokenize(tweet.replace("’", "'").replace("…", "..."))
    normTweet = " ".join([normalizeToken(token) for token in tokens])

    normTweet = normTweet.replace("cannot ", "can not ").replace("n't ", " n't ").replace("n 't ", " n't ").replace("ca n't", "can't").replace("ai n't", "ain't")
    normTweet = normTweet.replace("'m ", " 'm ").replace("'re ", " 're ").replace("'s ", " 's ").replace("'ll "," 'll ").replace("'d ", " 'd ").replace("'ve ", " 've ")
    normTweet = normTweet.replace(" p . m .", "  p.m.").replace(" p . m ", " p.m ").replace(" a . m ."," a.m.").replace(" a . m "," a.m ")
    normTweet = re.sub(r",([0-9]{2,4}) , ([0-9]{2,4})", r",\1,\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3}) / ([0-9]{2,4})", r"\1/\2", normTweet)
    normTweet = re.sub(r"([0-9]{1,3})- ([0-9]{2,4})", r"\1-\2", normTweet)
    normTweet = normTweet.lower()
    return " ".join(normTweet.split())


from sklearn.model_selection import train_test_split

# this method just for splitting
def splitting_method(df_, name1 ,name2, test_size = 0.5):
  y = pd.DataFrame(df_, columns = ["label"])
  X = pd.DataFrame(df_, columns = ['sentence'])

  X_train, X_test ,y_train, y_test = train_test_split(X, y, test_size=test_size, shuffle=True, random_state=105)

  df_t = pd.DataFrame(X_train, columns = ['sentence'])
  df_yt = pd.DataFrame(y_train, columns = ['label'])

  train_data = pd.concat([df_t, df_yt], axis=1)
  train_data.to_csv(name1+".csv", index = False,)

  df_xtest = pd.DataFrame(X_test, columns = ['sentence'])
  df_ytest = pd.DataFrame(y_test, columns = ['label'])

  test_data = pd.concat([df_xtest, df_ytest], axis=1)
  # print(test_data.isnull().sum())
  if test_size != 0.5:

    test_data = test_data.drop_duplicates('sentence')

  test_data.to_csv(name2+".csv", index = False,)

  # return train_data
  return test_data


In [ ]:
if torch.cuda.is_available():
    device = torch.device("cuda:0")
    print('There are %d GPU(s) available.' % torch.cuda.device_count())
    print('We will use the GPU:', torch.cuda.get_device_name(0))

else:
    print('No GPU available, using the CPU instead.')
    device = torch.device("cpu")


In [ ]:
# CHECKPOINT
CHECKPOINT = "Youssef320/LSTM-finetuned-50label-15epoch"             #change this to the path of your target hugging face model

from transformers import AutoModelForSequenceClassification

model = AutoModelForSequenceClassification.from_pretrained(
    CHECKPOINT,
    num_labels = 50,   #should be changed with another dataset
    output_attentions = False,
    output_hidden_states = False,
    ignore_mismatched_sizes=True,
)

model.to(device)

In [ ]:
# DATA loader fun
def Data_to_dataloader(File_name):

  df = pd.read_csv(File_name)

  print('Number of training sentences: {:,}\n'.format(df.shape[0]))
  # print('Number of dev sentences: {:,}\n'.format(df_dev.shape[0]))
  df['sentence']  = df.sentence.apply(normalizeTweet)
  df.dropna()
  # df_dev['sentence']  = df_dev.sentence.apply(normalizeTweet)
  # df_dev.dropna()



  # Get the lists of sentences and their labels.
  sentences = df.sentence.values
  labels = df.label.values
  # sentences_dev = df_dev.sentence.values
  # labels_dev = df_dev.label.values
  tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)


  input_ids = []
  # input_ids_dev = []

  for sent in sentences:
    encoded_sent = tokenizer.encode(sent)
    input_ids.append(encoded_sent)

  # for sent_dev in sentences_dev:
  #   encoded_sent_dev = tokenizer.encode(sent_dev)
  #   input_ids_dev.append(encoded_sent_dev)


  MAX_LEN = 64
  #MAX_LEN = 128
  print('\nPadding/truncating all sentences to %d values...' % MAX_LEN)
  print('\nPadding token: "{:}", ID: {:}'.format(tokenizer.pad_token, tokenizer.pad_token_id))
  input_ids = pad_sequences(input_ids, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  # input_ids_dev = pad_sequences(input_ids_dev, maxlen=MAX_LEN, dtype="long", value=0, truncating="post", padding="post")
  print('\nDone.')
  # Create attention masks


  attention_masks = []
  # attention_masks_dev = []
  for sent in input_ids:
      att_mask = [int(token_id > 0) for token_id in sent]
      attention_masks.append(att_mask)

  # for sent_dev in input_ids_dev:
  #     att_mask_dev = [int(token_id > 0) for token_id in sent_dev]
  #     attention_masks_dev.append(att_mask_dev)


  train_inputs = input_ids
  # validation_inputs = input_ids_dev

  train_labels = labels
  print("train_labels: ",set(train_labels))

  # validation_labels = labels_dev
  # print("validation_labels: ",set(validation_labels))

  train_masks = attention_masks
  # validation_masks = attention_masks_dev

  train_inputs = torch.tensor(train_inputs)
  # validation_inputs = torch.tensor(validation_inputs)

  train_labels = torch.tensor(train_labels)
  print("train_labels: ",train_labels)

  # validation_labels = torch.tensor(validation_labels)
  # print("validation_labels: ",validation_labels)

  train_masks = torch.tensor(train_masks)
  # validation_masks = torch.tensor(validation_masks)


  batch_size = 64
  # Create the DataLoader for our training set.
  train_data = TensorDataset(train_inputs, train_masks, train_labels)
  train_sampler = RandomSampler(train_data)
  train_dataloader = DataLoader(train_data, sampler=train_sampler, batch_size=batch_size)
  # Create the DataLoader for our validation set.
  # validation_data = TensorDataset(validation_inputs, validation_masks, validation_labels)
  # validation_sampler = SequentialSampler(validation_data)
  # validation_dataloader = DataLoader(validation_data, sampler=validation_sampler, batch_size=batch_size)

  return train_dataloader

In [ ]:
# change the paths to your files
train_dataloader = Data_to_dataloader('/mnt/azureml/cr/j/ec6c81310066482c9f2fcc2c79063234/exe/wd/TopLabeledTrain50V3.csv')
validation_dataloader = Data_to_dataloader('/mnt/azureml/cr/j/ec6c81310066482c9f2fcc2c79063234/exe/wd/TopLabeledTest50V3.csv')

In [ ]:
def train_fun(epoch_i, train_dataloader, model):

  optimizer = AdamW(model.parameters(),
                    lr = 2e-5,
                    eps = 1e-8
                  )
  total_steps = len(train_dataloader) * epochs
  scheduler = get_linear_schedule_with_warmup(optimizer,
                                              num_warmup_steps = 0, # Default value in run_glue.py
                                              num_training_steps = total_steps)


  # This training code is based on the `run_glue.py` script here:
  # https://github.com/huggingface/transformers/blob/5bfcd0485ece086ebcbed2d008813037968a9e58/examples/run_glue.py#L128
  seed_val = 42
  random.seed(seed_val)
  np.random.seed(seed_val)
  torch.manual_seed(seed_val)
  torch.cuda.manual_seed_all(seed_val)
  # Store the average loss after each epoch so we can plot them.
  loss_values = []

  # ========================================
  #               Training
  # ========================================
  # Perform one full pass over the training set.
  print("")
  print('======== Epoch {:} / {:} ========'.format(epoch_i + 1, epochs))
  print('Training...')
  # Measure how long the training epoch takes.
  t0 = time.time()
  # Reset the total loss for this epoch.
  total_loss = 0
  # Put the model into training mode. Don't be mislead--the call to
  # `train` just changes the *mode*, it doesn't *perform* the training.
  # `dropout` and `batchnorm` layers behave differently during training
  # vs. test (source: https://stackoverflow.com/questions/51433378/what-does-model-train-do-in-pytorch)
  model.train()

  best_loss = float('inf')  # Initialize best loss to infinity

  for step, batch in enumerate(train_dataloader):

      if step % 40 == 0 and not step == 0:
          elapsed = format_time(time.time() - t0)
          print('  Batch {:>5,}  of  {:>5,}.    Elapsed: {:}.'.format(step, len(train_dataloader), elapsed))

      b_input_ids = batch[0].to(device)
      b_input_mask = batch[1].to(device)
      b_labels = batch[2].to(device)
      # Always clear any previously calculated gradients before performing a
      # backward pass. PyTorch doesn't do this automatically because
      # accumulating the gradients is "convenient while training RNNs".
      # (source: https://stackoverflow.com/questions/48001598/why-do-we-need-to-call-zero-grad-in-pytorch)
      model.zero_grad()
      # The documentation for this `model` function is here:
      # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
      outputs = model(b_input_ids,
                  token_type_ids=None,
                  attention_mask=b_input_mask,
                  labels=b_labels)

      loss = outputs[0]
      total_loss += loss.item()
      #loss = calc_loss(outputs[1],b_labels)
      #total_loss += loss
      loss.backward()
      torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
      optimizer.step()
      scheduler.step()

  avg_train_loss = total_loss / len(train_dataloader)
  losses.append(avg_train_loss)
  loss_values.append(avg_train_loss)
  print("")
  print("  Average training loss: {0:.2f}".format(avg_train_loss))
  print("  Training epoch took: {:}".format(format_time(time.time() - t0)))

  # Save the model if the current loss is the best so far
  if avg_train_loss < best_loss:
      best_loss = avg_train_loss
      name_save = ' '
      torch.save(model, name_save)

  return loss_values

In [ ]:
def macro_score_f1(true_labels,predicted_labels):
    best_labels = []
    for i,label in enumerate(true_labels):
        flag = False
        for l in predicted_labels[i]:
            if l == label:
                best_labels.append(l)
                flag = True
        if not flag: best_labels.append(predicted_labels[i][0])
    #print(len(best_labels))
    #print(len(true_labels))
    macroF1Score = f1_score(true_labels, best_labels, average='macro')
    return macroF1Score


In [ ]:
def weighted_score_f1(true_labels,predicted_labels):
    best_labels = []
    for i,label in enumerate(true_labels):
        flag = False
        for l in predicted_labels[i]:
            if l == label:
                best_labels.append(l)
                flag = True
        if not flag: best_labels.append(predicted_labels[i][0])
    #print(len(best_labels))
    #print(len(true_labels))
    macroF1Score = f1_score(true_labels, best_labels, average='weighted')
    return macroF1Score

In [ ]:
def test_fun(validation_dataloader, model):
    # ========================================
    #               Validation
    # ========================================
    # After the completion of each training epoch, measure our performance on
    # our validation set.
    print("")
    print("Running Validation...")
    t0 = time.time()
    model.eval()
    eval_loss, eval_accuracy = 0, 0
    nb_eval_steps, nb_eval_examples = 0, 0

    predictions, true_labels = [], []

    for batch in validation_dataloader:
        batch = tuple(t.to(device) for t in batch)
        b_input_ids, b_input_mask, b_labels = batch

        with torch.no_grad():
            # The documentation for this `model` function is here:
            # https://huggingface.co/transformers/v2.2.0/model_doc/bert.html#transformers.BertForSequenceClassification
            outputs = model(b_input_ids,
                            token_type_ids=None,
                            attention_mask=b_input_mask)

        logits = outputs[0]
        logits = logits.detach().cpu().numpy()
        label_ids = b_labels.to('cpu').numpy()
        tmp_eval_accuracy = flat_accuracy(logits, label_ids)
        eval_accuracy += tmp_eval_accuracy
        nb_eval_steps += 1

        predictions.append(logits)
        true_labels.append(label_ids)
    Accs.append(eval_accuracy/nb_eval_steps)
    print('    DONE.')
    print("  Accuracy: {0:.2f}".format(eval_accuracy/nb_eval_steps))
    print("  Validation took: {:}".format(format_time(time.time() - t0)))

    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argmax(flat_predictions, axis=1).flatten()
    flat_true_labels = [item for sublist in true_labels for item in sublist]

    # Calculate the confusion matrix
    conf_matrix = confusion_matrix(flat_true_labels, flat_predictions)

    # Plot the confusion matrix
    plt.figure(figsize=(25, 20))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues')
    plt.title('Confusion Matrix')
    plt.xlabel('Predicted Labels')
    plt.ylabel('True Labels')
    plt.show()

    # Print the classification report
    classif_rep = classification_report(flat_true_labels, flat_predictions, digits=5)
    print("Classification Report:")
    print(classif_rep)
    ############################################################################################
    flat_predictions = [item for sublist in predictions for item in sublist]
    flat_predictions = np.argsort(flat_predictions, axis=1)[:, -3:]  # Get the indices of top 3 predictions
    flat_true_labels = [item for sublist in true_labels for item in sublist]

    top3_accuracy = np.mean([1 if true_label in pred_labels else 0 for true_label, pred_labels in zip(flat_true_labels, flat_predictions)])
    print("  Top 3 Accuracy: {0:.2f}".format(top3_accuracy))
    ##############################################################################
    #print(flat_predictions)
    #print(flat_true_labels)
    #x = f1_score(flat_true_labels, flat_predictions, average='macro')
    top3_f1 = macro_score_f1(flat_true_labels,flat_predictions)
    print(f"top 3 macro f1 score : {top3_f1}")
    top3_weightedf1 = weighted_score_f1(flat_true_labels,flat_predictions)
    print(f"top 3 weighted f1 score : {top3_weightedf1}")
    f1score.append(top3_f1)
    top3acc.append(top3_accuracy)
    Top3_weightedf1.append(top3_weightedf1)

In [ ]:
Accs = []
losses = []
f1score = []
top3acc = []
Top3_weightedf1  = []
num_epochs = 15      #change number of epochs if you want
for epoch_i in range(0, num_epochs):
  train_fun(epoch_i, train_dataloader, model)

  print('Data validation>>>')
  test_fun(validation_dataloader, model)

In [ ]:
# plot the accuracy per epoch
# Assuming you have a list called 'accuracy_list'

# Create a list of indices (epochs)
epochs = list(range(len(Accs)))

# Plot the accuracy values against the epochs
plt.plot(epochs, Accs, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.title('Accuracy per Epoch')
plt.grid(True)
plt.show()

In [ ]:
#plot the total loss per epoch

epochs = list(range(len(losses)))

# Plot the accuracy values against the epochs
plt.plot(epochs, losses, marker='o')
plt.xlabel('Epoch')
plt.ylabel('loss')
plt.title('loss per Epoch')
plt.grid(True)
plt.show()

In [ ]:
#plot the f1score per epoch

epochs = list(range(len(f1score)))

# Plot the accuracy values against the epochs
plt.plot(epochs, f1score, marker='o')
plt.xlabel('Epoch')
plt.ylabel('Macro avg top 1 f1 score')
plt.title('f1 score per Epoch')
plt.grid(True)
plt.show()

# loading the best model and pushing it to hugging face

In [ ]:
model = torch.load("path to your best model. will be in your directory with name Best_Model.pt")

In [ ]:
!huggingface-cli login --token $"put here your token key, generate it from hugging face website"

In [ ]:
model.push_to_hub("give a meaningful name here about your model")


In [ ]:
tokenizer = AutoTokenizer.from_pretrained(CHECKPOINT)


In [ ]:
tokenizer.push_to_hub("give a meaningful name here about your model")